# Mini Project 3: COVID-19 Data Analysis and Machine Learning

#### Created by Group 7 - Kamilla, Jeanette, Juvena

## Objective

This assignment aims to develop practical skills in data analysis, visualization, and machine learning using real-world COVID-19 data. The project focuses on exploring global pandemic-related indicators to uncover trends, build predictive models, and apply both supervised and unsupervised learning techniques using Python.

Before we begin analyzing the COVID-19 dataset, we need to import a few essential Python libraries that will help us manipulate the data, build models, and visualize our findings:

- **Pandas**: This is a powerful library used to handle and manipulate data in tables (called DataFrames).
- **NumPy**: It helps with numerical operations, especially when we work with arrays or need to do math.
- **Matplotlib** and **Seaborn**: These are popular libraries for creating visual charts and graphs. We'll use them to help us understand the data better by seeing it.
- **SciPy (stats module)**: This gives us access to statistical tools like checking if data is normally distributed.

We'll also configure default styles for our plots to ensure they're clean, visually appealing, and easy to interpret.

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import metrics
from scipy.spatial.distance import cdist
from sklearn import preprocessing as prep
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Set plot styles for better visualization
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

--------

# 1. Data wrangling and exploration

### Load the Data

Now that we have our tools ready, the next step is to load the COVID-19 dataset into Python so we can start analyzing it.

In this case, we’re working with a single dataset:

- **OWID COVID-19 Latest Data**: a CSV file that contains country-level information on cases, deaths, vaccinations, testing, and various socioeconomic indicators.

We'll use Pandas to read the CSV file and store it as a DataFrame. To make our code cleaner and reusable, we'll define a simple function that loads the data and performs some initial checks. This way, we can easily reload or replace the dataset if needed in future steps.

In [ ]:
# File paths for the wine quality datasets
dataset_covid = 'Dataset/owid-covid-latest.csv'

# Function to load the Excel files
def load_csv_to_dataframe(file_path):
    # Reads the Excel file and skips the first row if it contains a description or title
    df = pd.read_csv(file_path)
    return df

# Load datasets
print("..Loading COVID-19 dataset")
df_covid = load_csv_to_dataframe(dataset_covid)

..Loading COVID-19 datasets


### Explore the Data

After loading the dataset, we want to explore it to understand what kind of information it contains and how it's structured.

To do this, we can use several helpful Pandas functions such as `shape`, `types`, `info()`, `head()`, `tail()`, `sample()`, `describe()` and `isnull().sum()`. These functions will give us insights into the number of rows and columns, the data types of each column, a summary of the data, and any missing values. 

This exploration is crucial as it helps us identify potential issues or areas that need further cleaning or transformation before we proceed with our analysis. 

In [7]:
# Check the shape of the DataFrame (rows, columns)
df_covid.shape

(247, 67)

In [ ]:
# Display the types of attributes (colum names) in the DataFrame
df_covid.dtypes

iso_code                                    object
continent                                   object
location                                    object
last_updated_date                           object
total_cases                                float64
                                            ...   
population                                 float64
excess_mortality_cumulative_absolute       float64
excess_mortality_cumulative                float64
excess_mortality                           float64
excess_mortality_cumulative_per_million    float64
Length: 67, dtype: object

In [ ]:
# Gives an overview of the DataFrame
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 67 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   iso_code                                    247 non-null    object 
 1   continent                                   235 non-null    object 
 2   location                                    247 non-null    object 
 3   last_updated_date                           247 non-null    object 
 4   total_cases                                 246 non-null    float64
 5   new_cases                                   242 non-null    float64
 6   new_cases_smoothed                          242 non-null    float64
 7   total_deaths                                246 non-null    float64
 8   new_deaths                                  243 non-null    float64
 9   new_deaths_smoothed                         243 non-null    float64
 10  total_cases_pe

In [9]:
# Display the first 5 rows of the DataFrame
df_covid.head()

,iso_code,continent,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2024-08-04,235214.0,0.0,0.000,7998.0,0.0,0.0,...,NaN,37.746,0.50,64.83,0.511,4.112877e+07,NaN,NaN,NaN,NaN
1,OWID_AFR,NaN,Africa,2024-08-04,13145380.0,36.0,5.143,259117.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,1.426737e+09,NaN,NaN,NaN,NaN
2,ALB,Europe,Albania,2024-08-04,335047.0,0.0,0.000,3605.0,0.0,0.0,...,51.2,NaN,2.89,78.57,0.795,2.842318e+06,NaN,NaN,NaN,NaN
3,DZA,Africa,Algeria,2024-08-04,272139.0,18.0,2.571,6881.0,0.0,0.0,...,30.4,83.741,1.90,76.88,0.748,4.490323e+07,NaN,NaN,NaN,NaN
4,ASM,Oceania,American Samoa,2024-08-04,8359.0,0.0,0.000,34.0,0.0,0.0,...,NaN,NaN,NaN,73.74,NaN,4.429500e+04,NaN,NaN,NaN,NaN


In [10]:
# Display the last 5 rows of the DataFrame
df_covid.tail()

,iso_code,continent,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
242,WLF,Oceania,Wallis and Futuna,2024-08-04,3760.0,0.0,0.000,9.0,0.0,0.000,...,NaN,NaN,NaN,79.94,NaN,1.159600e+04,NaN,NaN,NaN,NaN
243,OWID_WRL,NaN,World,2024-08-14,775866783.0,47169.0,6738.429,7057132.0,815.0,116.429,...,34.635,60.130,2.705,72.58,0.737,7.975105e+09,NaN,NaN,NaN,NaN
244,YEM,Asia,Yemen,2024-08-04,11945.0,0.0,0.000,2159.0,0.0,0.000,...,29.200,49.542,0.700,66.12,0.470,3.369661e+07,NaN,NaN,NaN,NaN
245,ZMB,Africa,Zambia,2024-08-04,349842.0,18.0,2.571,4077.0,0.0,0.000,...,24.700,13.938,2.000,63.89,0.584,2.001767e+07,NaN,NaN,NaN,NaN
246,ZWE,Africa,Zimbabwe,2024-08-04,266386.0,0.0,0.000,5740.0,0.0,0.000,...,30.700,36.791,1.700,61.49,0.571,1.632054e+07,NaN,NaN,NaN,NaN


In [11]:
# Display a random sample of 5 rows from the DataFrame
df_covid.sample(5)

,iso_code,continent,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
190,SPM,North America,Saint Pierre and Miquelon,2024-08-04,3426.0,0.0,0.000,2.0,0.0,0.0,...,NaN,NaN,NaN,81.07,NaN,5885.0,NaN,NaN,NaN,NaN
75,FIN,Europe,Finland,2024-08-04,1499712.0,0.0,0.000,11466.0,0.0,0.0,...,22.6,NaN,3.28,81.91,0.938,5540745.0,NaN,NaN,NaN,NaN
106,IMN,Europe,Isle of Man,2024-08-04,38008.0,0.0,0.000,116.0,0.0,0.0,...,NaN,NaN,NaN,81.40,NaN,84534.0,NaN,NaN,NaN,NaN
232,GBR,Europe,United Kingdom,2024-08-04,24974629.0,3400.0,485.714,232112.0,0.0,0.0,...,24.7,NaN,2.54,81.32,0.932,67508936.0,NaN,NaN,NaN,NaN
153,NPL,Asia,Nepal,2024-08-04,1003450.0,0.0,0.000,12031.0,0.0,0.0,...,37.8,47.782,0.30,70.78,0.602,30547586.0,NaN,NaN,NaN,NaN


In [ ]:
# Gives summary statistics for all numerical columns in the dataset
df_covid.describe()

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
count,2.460000e+02,242.000000,242.000000,2.460000e+02,243.000000,243.00000,246.000000,242.000000,242.000000,246.000000,...,145.000000,96.000000,173.000000,231.000000,190.000000,2.470000e+02,0.0,0.0,0.0,0.0
mean,1.336634e+07,885.607438,126.515355,1.198689e+05,14.032922,2.00472,203988.255797,22.204909,3.172136,1271.427736,...,32.909897,50.788844,3.097012,73.660866,0.722500,1.307656e+08,NaN,NaN,NaN,NaN
std,6.568130e+07,4854.786157,693.540908,5.747240e+05,92.179347,13.16853,200456.902140,82.962646,11.851812,1322.697453,...,13.621757,32.124848,2.555777,7.405725,0.149398,6.684333e+08,NaN,NaN,NaN,NaN
min,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,7.700000,1.188000,0.100000,53.280000,0.394000,4.700000e+01,NaN,NaN,NaN,NaN
25%,2.750950e+04,0.000000,0.000000,1.837500e+02,0.000000,0.00000,21257.766500,0.000000,0.000000,144.808250,...,22.600000,20.482000,1.300000,69.545000,0.603000,4.294955e+05,NaN,NaN,NaN,NaN
50%,2.320985e+05,0.000000,0.000000,2.205500e+03,0.000000,0.00000,135384.895000,0.000000,0.000000,877.689000,...,33.100000,49.690500,2.500000,75.050000,0.740000,5.970430e+06,NaN,NaN,NaN,NaN
75%,1.703974e+06,5.500000,0.785500,1.938850e+04,0.000000,0.00000,340625.300000,0.232000,0.033250,2032.222000,...,41.300000,82.686750,4.200000,79.285000,0.828750,2.895671e+07,NaN,NaN,NaN,NaN
max,7.758668e+08,47169.000000,6738.429000,7.057132e+06,815.000000,116.42900,763598.600000,672.437000,96.062000,6601.110000,...,78.100000,100.000000,13.800000,86.750000,0.957000,7.975105e+09,NaN,NaN,NaN,NaN


##### Summary of exploring the data

After exploring the dataframe, we found that it contains a large number of columns, many of which are not useful for our analysis or modeling goals. While some columns provide valuable information (like total cases, deaths, and vaccination rates), others are either redundant, mostly empty, or irrelevant.

This highlights the need for a thorough data cleaning step to remove unnecessary columns, handle missing values, and focus only on the most relevant features for our machine learning tasks.

### Clean the Data

After loading and exploring the data, we need to clean it to ensure that our analysis is accurate and meaningful. Data cleaning involves several steps, including: checking for missing values, removing duplicates, and converting data types.

In [15]:
# Check for missing values in the DataFrame
df_covid.isnull().sum()

iso_code                                     0
continent                                   12
location                                     0
last_updated_date                            0
total_cases                                  1
                                          ... 
population                                   0
excess_mortality_cumulative_absolute       247
excess_mortality_cumulative                247
excess_mortality                           247
excess_mortality_cumulative_per_million    247
Length: 67, dtype: int64

# 2. Supervised machine learning: linear regression

# 3. Supervised machine learning: classification

# 4. Unsupervised machine learning: clustering

# 5. Implementation of the models in a Streamlit application